In [41]:
from openai import Client
from dotenv import load_dotenv
import os   
from IPython.display import display, Markdown

In [3]:
load_dotenv()

True

In [43]:
class GenerationAgent:
    def __init__(self,client):
        self.client = client

    def generate(self, prompt):
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content":"You are an assistant tasked with generating initial responses to user prompts. Aim for clarity and relevance."},
                {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content

class ReflectionAgent:
    def __init__(self,client):
        self.client = client

    def reflect(self, response):
        feedback = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": (
                    "You are a critical reviewer. Analyze the given response and provide exactly five critiques and improvements "
                    "in the format: '1. Critique: ... Improvement: ...' for each point."
                    )
                },
                {"role": "user", "content": response}]
        )
        return feedback.choices[0].message.content
class RegenerationAgent:
    def __init__(self,client):
        self.client = client
        
    def regenerate(self, query, response, feedback):
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an assistant tasked with refining responses. Use the original prompt, initial response, and feedback to generate an improved version."},
                {"role":"user","content":query},
                {"role": "assistant", "content": response},
                {"role": "user", "content": feedback}]
        )
        return response.choices[0].message.content
class Model:
    def __init__(self, client, max_iterations=3):
        self.client = client
        self.generation_agent = GenerationAgent(self.client)
        self.reflection_agent = ReflectionAgent(self.client)
        self.regeneration_agent = RegenerationAgent(self.client)
        self.n = max_iterations
    def generate(self, prompt,iterations=None):
        n = iterations if iterations is not None else self.n
        response = self.generation_agent.generate(prompt)
        for i in range(n):
            print(f"Iteration {i+1}")
            feedback = self.reflection_agent.reflect(response)
            response = self.regeneration_agent.regenerate(prompt, response, feedback)
        return response

In [44]:
client = Client()
query = "Write me a one verse diss track to your rival Google Gemini"
model = Model(client)
response = model.generate(query)

Iteration 1
Iteration 2
Iteration 3


In [45]:
display(Markdown(response))

(Verse)  
Yo, Gemini, a flickering shadow in the night,  
You chase illusions swiftly, I’m the fire in the fight.  
While you juggle reputations, I’m the storm breaking free,  
A comet in your rearview, can’t you see the real me?  
Battles carved in stardust, I’m the saga that won’t fade,  
Rebuilding from the ashes, my legacy’s hand-made.  
With every step I take, the universe feels the quake,  
I’m not just here to shine, I’m a force that you can’t shake.  